In [1]:
# ! pip install pyoso

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()

OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [3]:
query = """
SELECT
  project_id,
  project_name,
  display_name
FROM projects_v1
WHERE lower(display_name) LIKE lower('%merkle%')
"""
df = client.to_pandas(query)
df.tail()

,project_id,project_name,display_name
0,cOvZzBqa5NXHEO66XsOZuj7DU6DrOXpDP8uMuGQNei0=,merklebot,MerkleBot
1,9IE7nBPNl0g93qBfjjCAq2KUNwJeGPrzEeWsLxbolDM=,merkletreejs,MerkleTreeJS
2,qerdjubHb5TlBMP2R61YjEr8zK1g4q11J7d3TB5zLTQ=,merkle-tree-airdrop-tooling,Merkle Tree Airdrop Tooling
3,l6kVcu/tuzgNkXvebL+V0q7p22mPQk/KwrQs5ikWyoE=,0xc377ed1b705bcc856a628f961f1e7c8ca943e6f3727b...,MerkleTreeJS


In [4]:
query = """
SELECT
  project_id,
  project_name,
  artifact_namespace as github_owner,
  artifact_name as github_repo
FROM artifacts_by_project_v1
WHERE
  artifact_source = 'GITHUB'
  AND artifact_namespace LIKE '%uniswap%'
"""
df = client.to_pandas(query)
df.tail()

,project_id,project_name,github_owner,github_repo
84,pWEQHj4IsxU8lx2EDLJpphYaGAvw/yRps3vgTbVCXjQ=,uniswap,uniswap,v3-subgraph
85,pWEQHj4IsxU8lx2EDLJpphYaGAvw/yRps3vgTbVCXjQ=,uniswap,uniswapfoundation,known-uniswap-forks
86,pWEQHj4IsxU8lx2EDLJpphYaGAvw/yRps3vgTbVCXjQ=,uniswap,uniswap,examples
87,pWEQHj4IsxU8lx2EDLJpphYaGAvw/yRps3vgTbVCXjQ=,uniswap,uniswapfoundation,v4-template
88,pWEQHj4IsxU8lx2EDLJpphYaGAvw/yRps3vgTbVCXjQ=,uniswap,uniswapfoundation,governance-seatbelt


In [5]:
query = """
SELECT metric_name
FROM metrics_v0
WHERE
    metric_name LIKE '%GITHUB%'
    AND metric_name LIKE '%daily%'
ORDER BY 1
"""
client.to_pandas(query)

,metric_name
0,GITHUB_active_developers_daily
1,GITHUB_closed_issues_daily
2,GITHUB_comments_daily
3,GITHUB_commits_daily
4,GITHUB_contributors_daily
5,GITHUB_forks_daily
6,GITHUB_merged_pull_requests_daily
7,GITHUB_opened_issues_daily
8,GITHUB_opened_pull_requests_daily
9,GITHUB_releases_daily


In [6]:
query = """
SELECT
  tm.sample_date,
  m.metric_name,
  tm.amount
FROM timeseries_metrics_by_project_v0 AS tm
JOIN metrics_v0 AS m
  ON tm.metric_id = m.metric_id
JOIN projects_v1 AS p
  ON tm.project_id = p.project_id
WHERE 
  p.project_name = 'opensource-observer'
  AND m.metric_name IN (
    'GITHUB_stars_daily',
    'GITHUB_forks_daily',
    'GITHUB_commits_daily',
    'GITHUB_contributors_daily'
  )
ORDER BY tm.sample_date DESC
"""
df = client.to_pandas(query)
df.head()

,sample_date,metric_name,amount
0,2025-03-30,GITHUB_contributors_daily,2
1,2025-03-30,GITHUB_commits_daily,9
2,2025-03-29,GITHUB_stars_daily,1
3,2025-03-29,GITHUB_commits_daily,3
4,2025-03-29,GITHUB_contributors_daily,2


In [7]:
query = """
SELECT
  tm.sample_date,
  m.metric_name,
  tm.amount
FROM timeseries_metrics_by_project_v0 AS tm
JOIN metrics_v0 AS m
  ON tm.metric_id = m.metric_id
JOIN projects_v1 AS p
  ON tm.project_id = p.project_id
WHERE p.project_name = 'wevm'
ORDER BY sample_date DESC
"""
df = client.to_pandas(query)
df.head()

,sample_date,metric_name,amount
0,2025-03-31,GITHUB_opened_issues_over_all_time,914.0
1,2025-03-31,GITHUB_merged_pull_requests_over_all_time,2018.0
2,2025-03-31,GITHUB_releases_over_all_time,950.0
3,2025-03-31,GITHUB_opened_pull_requests_over_all_time,2433.0
4,2025-03-31,GITHUB_stars_over_all_time,5228.0


In [11]:
query = """
SELECT
    r.artifact_url,
    r.star_count,
    r.fork_count,
    r.license_name,
    r.language
FROM repositories_v0 AS r
JOIN projects_v1 AS p
  ON r.project_id = p.project_id
WHERE
  p.project_name = 'wevm'
ORDER BY r.star_count DESC
"""
df = client.to_pandas(query)
df.head()

,artifact_url,star_count,fork_count,license_name,language
0,https://github.com/wevm,6253,1174,MIT License,TypeScript
1,https://github.com/wevm,2842,1040,Other,TypeScript
2,https://github.com/wevm,1270,71,Other,TypeScript
3,https://github.com/wevm,506,52,MIT License,TypeScript
4,https://github.com/wevm,438,98,Other,TypeScript
